In [1]:
import tensorflow as tf

In [2]:
import tensorflow_hub as hub

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Layer, concatenate, Concatenate, Reshape, Conv2D, Conv1D
from tensorflow.keras.models import Model, load_model, model_from_json
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn import datasets
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
from flask import Flask, request, render_template
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [4]:
FILE_DIR = '/Users/i509787/OneDrive - SAP SE/from_HP/AI_COVID/text_processing/package_app_v3/request_classifier'
module_url = os.path.join(FILE_DIR, "tf_sent_encoder_2")

session = tf.Session()
K.set_session(session)

embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value


def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
model_use = Model(inputs=[input_text], outputs=embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
session.run(tf.global_variables_initializer())

In [6]:
session.run(tf.tables_initializer())

In [7]:
model_use.predict(np.array(['hello', '']))

array([[-0.02554519,  0.00690543, -0.00611844, ...,  0.03180381,
        -0.09227505, -0.00998536],
       [ 0.00510745, -0.01180556, -0.0489458 , ...,  0.04829174,
        -0.07290252, -0.05681312]], dtype=float32)

In [8]:
model_use.predict(np.array(['hello', ''])).shape

(2, 512)

In [9]:
def cal_euclidean_dist(v1, v2):  
    temp = v1 - v2
    sum_sq = np.dot(temp.T, temp)
    return np.sqrt(sum_sq)
    
def get_similarity(text1, text2):
    emb1, emb2 = model_use.predict(np.array([text1, text2]))
    return cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]

def get_distance(text1, text2):
    emb1, emb2 = model_use.predict(np.array([text1, text2]))
    return cal_euclidean_dist(emb1, emb2)

def get_sim_and_dist(text1, text2):
    emb1, emb2 = model_use.predict(np.array([text1, text2]))
    sim = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
    dist = cal_euclidean_dist(emb1, emb2)
    return sim, dist

In [10]:
print (get_similarity('python engineer', 'java engineer'))
print (get_distance('python engineer', 'java engineer'))

0.76429963
0.6865865


In [11]:
#SOC analyst with marketing manager 1

get_sim_and_dist('''We are looking for an   and dedicated     to work on daily security monitoring of alerts and various security systems. You will lead and mentor broader SOC operational teams to achieve excellence in day to day operations. 
You are the  ; supporting various security issues, offering expert advice to internal and external stakeholders across the entire organisation. 
You’ll also need
Experience in delivering   within a SOC environment.
Commercial experience on  , 
Experience in   and security event investigation.
Strong understanding of   and incident handling processes.
Exposure to   or Azure Security management highly desirable.
Experience in liaising with application, architecture, development, and information security teams to ensure the business meets IT Security standards.''', 
'''Developed new promotional campaign by bundling products together, increasing sales revenue by 13% with projected year-on-sales growth of $400K
Negotiated new contracts with vendors, saving $120K for the marketing budget
Recruited, hired, and trained over 30 marketing and sales specialists, improving overall sales targets by 24% while maintaining 86% in employee retention
Referenced three times in articles for exceptional advertising campaigns
Directed launch of 12 new product lines, with total annual revenue of $1.3B''')

(0.35955423, 1.1317648)

In [12]:
#SOC analyst with cloud architect 2
get_sim_and_dist('''We are looking for an   and dedicated     to work on daily security monitoring of alerts and various security systems. You will lead and mentor broader SOC operational teams to achieve excellence in day to day operations. 
You are the  ; supporting various security issues, offering expert advice to internal and external stakeholders across the entire organisation. 
You’ll also need
Experience in delivering   within a SOC environment.
Commercial experience on  , 
Experience in   and security event investigation.
Strong understanding of   and incident handling processes.
Exposure to   or Azure Security management highly desirable.
Experience in liaising with application, architecture, development, and information security teams to ensure the business meets IT Security standards.''', 
'''Work with DevOps to make sure there are tools in place for monitoring the system, and log management to effectively handle incidents
Perform service delivery platform benchmarks and develop framework for cloud implementations
Provide effective leadership, development, coaching and guidance to the team, and Lines of Business of SAP
Create, evolve, mature and execute a cloud migration plan that includes detailed road map and risk assessment and mitigation approach
Provide technical leadership to evolve, mature and migrate mission management application to C2S
Provide a globally unique process for interacting with partners regarding VSOE compliant pricing, committed delivery dates and handover – including Partner onboarding
Partner with infrastructure operations, platform optimization, and datacenter operation teams to scope, define, size and execute application onboarding, migration & consolidation initiatives''')

(0.7586067, 0.69482857)